In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob, os, shutil
import statistics
import json 
import requests
from difflib import SequenceMatcher
import sys
#sys.path.insert(1, '/home/marinedjaffardjy/Documents/wf_features/src/parsing')
#import parsing_snkmk as ps
#import get_characteristics as t
import json

In [2]:
def importing_json_files(file_wf):
    f_wf = open(file_wf) #informations for nf
    # returns JSON object as
    # a dictionary
    wf = json.load(f_wf)
    f_wf.close
    return wf

In [3]:
#importing the wf and auth dict (github info)
dict_nf = importing_json_files('json/wf_new_crawl_nextflow.json')
auth_nf = importing_json_files('json/author_clem_nf.json')
dict_snk = importing_json_files('json/wf_crawl_snakemake.json')
auth_snk = importing_json_files('json/author_clem_snk.json')


In [4]:
#importing the similarity groups
with open("/home/marinedjaffardjy/Documents/Code/Similarite_process/json/simil_groups_levenshtein_500.json") as f:
    groups_500 = json.load(f)
#treshold of 0.8, snakemake

Making the similarity dataframes

In [7]:
def sim_to_df_snk(groups_list, dict_wf): 
    groups_sim_df = pd.DataFrame(columns = ['nb_reuse', 'tools','nb_own', 'list_own', 'nb_contrib','list_contrib','nb_wf','list_wf'])
    for group in groups_list :
        list_own = []
        list_wf = []
        list_contrib = []
        for el in group :
            list_own.append(el['owner'])
            list_wf.append(el['wf_orig'])
            name_wf = el['wf_orig'].split('/')[0]+'/'+el['wf_orig'].split('/')[1]
            for contr in dict_wf[name_wf]['contributors']:
                list_contrib.append(contr)
        list_own = list(set(list_own))
        list_wf = list(set(list_wf))
        list_contrib = list(set(list_contrib))
        groups_sim_df = groups_sim_df.append({'nb_reuse' : len(group),
                                             'nb_own' : len(list_own),
                                             'tools' : group[0]['tools'],
                                             'list_own' : list_own ,
                                             'nb_wf' : len(list_wf),
                                             'list_wf' : list_wf,
                                             'list_contrib' : list_contrib,
                                             'nb_contrib' :len(list_contrib)}, ignore_index = True)
    return groups_sim_df

In [12]:
groups_500[2]

[497]

In [8]:
%%time
df_sim_500 = sim_to_df_snk(groups_500, dict_snk)

TypeError: 'int' object is not subscriptable

In [ ]:
def grouping_sim_df (df_sim_n):
    df_sim = df_sim_n.copy()
    vals = set(df_sim["nb_reuse"])
    tot_group = len(df_sim_n)
    tot_wf = sum(df_sim_n["nb_wf"])
    tot_own = sum(df_sim_n["nb_own"])

    df_sim_group = pd.DataFrame(columns=["nb_reuse","tool", "nb_proc","prop_proc","nb_own","prop_own","nb_wf","prop_wf"])
    for val in vals :
        sec_df = df_sim.loc[df_sim["nb_reuse"]==val]
        tools = []
        for el in sec_df["tools"]:
            tools.append(el)
        nb_proc = len(sec_df)
        prop_proc = nb_proc/tot_group
        nb_own = sum(sec_df["nb_own"])
        prop_own = nb_own/tot_own
        nb_wf = sum(sec_df["nb_wf"])
        prop_wf = nb_wf/tot_wf
        df_sim_group = df_sim_group.append({"nb_reuse":val,
                                            "tool":tools, 
                                            "nb_proc":nb_proc,
                                            "prop_proc":prop_proc*100.0,
                                            "nb_own":nb_own,
                                            "prop_own":prop_own*100.0,
                                            "nb_wf":nb_wf,
                                            "prop_wf":prop_wf*100.0}, ignore_index = True)
    
    return df_sim_group.sort_values(by=["nb_reuse"], ascending=True)




In [ ]:
df_groups_snk = grouping_sim_df(df_sim_snk)

In [ ]:
def sum_columns_df(df):
    dfret = pd.DataFrame(columns=["nb_reuse","tool", "nb_proc","prop_proc","nb_own","prop_own","nb_wf","prop_wf"])
    return dfret.append({"nb_reuse":list(df["nb_reuse"])[0],
                      "tool":list(df["tool"]), 
                      "nb_proc":df["nb_proc"].sum(),
                      "prop_proc":df["prop_proc"].sum(),
                      "nb_own":df["nb_own"].sum(),
                      "prop_own":df["prop_own"].sum(),
                      "nb_wf":df["nb_wf"].sum(),
                      "prop_wf":df["prop_wf"].sum()}, ignore_index=True)

In [ ]:
def display_proportions_figure(df_init):
    # découpage du wor
    df = df_init.iloc[0:9]
    i=9
    ranges = []
    while i+10 < len(df_init):
        ranges.append(str(df_init.iloc[i]["nb_reuse"])+"-"+str(df_init.iloc[i+10]["nb_reuse"]))
        sub_df = df_init.iloc[i:i+10]
        i+=10
        df = df.append(sum_columns_df(sub_df))
    ranges.append(str(df_init.iloc[i]["nb_reuse"])+"-"+str(df_init.iloc[len(df_init)-1]["nb_reuse"]))
    df = df.append(sum_columns_df(df_init.iloc[i:len(df_init)]))
    fig_4 = df.plot(x="nb_reuse", y=["prop_proc", "prop_wf", 'prop_own'], kind="bar", color = ['yellow','blue','red'], label = ['times used','workflows', 'owners'])
    fig_4.set_xticklabels(fig_4.get_xticklabels()[0:9]+ ranges, rotation=80, size=5)
    fig_4.set_xlabel('Number of times used/workflows used in/authors used by')
    fig_4.set_ylabel('Percent of processors')
    fig_4.legend(loc='best')
    fig_4.set_yscale('log')
    plt.show()
    plt.rcParams['figure.dpi'] = 500
    plt.rcParams['savefig.dpi'] = 500
    
    return fig_4

In [ ]:
# top 5 most used processors nextflow
df_sim_nf.sort_values(by=["nb_reuse"], ascending = False).head(5)

In [ ]:
fig_4_nf = display_proportions_figure(df_groups_nf)